In [1]:
def evaluate_ae_on_no_mtd_behavior(autoencoder: AutoEncoder, test_data):
    results = []
    for b, d in test_data.items():
        y_test = np.array([0 if b == Behavior.NORMAL else 1] * len(d))
        y_predicted = autoencoder.predict(d[:, :-1].astype(np.float32))

        acc, f1, conf_mat = calculate_metrics(y_test.flatten(), y_predicted.flatten().numpy())
        #res_dict[b] = f'{(100 * acc):.2f}%'
        results.append([b, f'{(100 * acc):.2f}%', f'{(100 * f1):.2f}%'])
    #labels = ["Behavior", "Accuracy", "F1-Score"] 
    #results = []
    #for b, a in res_dict.items():
        #results.append([b.value, res_dict[b]])
    print(tabulate(results, headers=["Behavior", "Accuracy", "F1-Score"], tablefmt="latex"))
    print(conf_mat)

NameError: name 'AutoEncoder' is not defined